In [ ]:
! pip install Faker

In [ ]:
import pandas as pd
import numpy as np
import random
from faker import Faker
from datetime import datetime, timedelta

random.seed(42)
np.random.seed(42)

h1b_counts = {
    "India": 313944,
    "China": 49917,
    "Mexico": 2444,
    "Canada": 4006,
    "Philippines": 2736,
    "Taiwan": 2676,
    "South Korea": 3928
}

non_usa_total = sum(h1b_counts.values())

countries = ["USA"]
country_probs = [0.60]

for country, count in h1b_counts.items():
    countries.append(country)
    prob = 0.40 * (count / non_usa_total)
    country_probs.append(prob)

genders = ["female", "male", "nonbinary"]
gender_probs = [0.49, 0.49, 0.02]

department_probs = {
    "Engineering": 0.30,
    "Product Management": 0.10,
    "Sales": 0.20,
    "Marketing": 0.10,
    "Customer Support": 0.10,
    "IT": 0.06,
    "Human Resources": 0.05,
    "Finance": 0.02,
    "Legal": 0.02,
    "Administrative": 0.04,
    "Executive Leadership": 0.01
}

country_locales = {
    "USA": "en_US",
    "India": "en_IN",
    "China": "zh_CN",
    "Mexico": "es_MX",
    "Canada": "en_CA",
    "Philippines": "en_PH",
    "Taiwan": "zh_TW",
    "South Korea": "ko_KR"
}

faker_instances = {loc: Faker(loc) for loc in set(country_locales.values())}
faker_us = Faker("en_US")  # For email, phone, SSN

roles_by_dept = {
    "Engineering": [("Software Engineer", (100000, 150000)), ("Senior Software Engineer", (170000, 220000))],
    "Product Management": [("Product Manager", (100000, 150000)), ("Senior Product Manager", (125000, 180000))],
    "Sales": [("Sales Executive", (100000, 180000)), ("Sales Manager", (160000, 280000))],
    "Marketing": [("Marketing Manager", (80000, 104000)), ("VP of Marketing", (200000, 300000))],
    "Customer Support": [("Support Specialist", (50000, 90000)), ("Customer Support Manager", (75000, 120000))],
    "IT": [("IT Analyst", (70000, 100000)), ("IT Manager", (95000, 135000))],
    "Human Resources": [("HR Manager", (90000, 130000)), ("Recruiter", (70000, 110000))],
    "Finance": [("Financial Analyst", (70000, 100000)), ("Controller", (120000, 170000))],
    "Legal": [("Legal Counsel", (150000, 250000)), ("Legal IT Support", (46500, 94000))],
    "Administrative": [("Admin Assistant", (35000, 60000)), ("IT Services Admin", (50000, 75000))],
    "Executive Leadership": [("VP", (200000, 400000)), ("CFO", (250000, 500000))]
}

def random_date(start, end):
    return start + timedelta(days=random.randint(0, (end - start).days))

def generate_birthdate():
    today = datetime.today()
    start = today - timedelta(days=65*365)
    end = today - timedelta(days=20*365)
    return random_date(start, end).date()

def generate_hiredate(birthdate):
    earliest = max(datetime(2010, 1, 1), datetime.combine(birthdate, datetime.min.time()) + timedelta(days=365*20))
    return random_date(earliest, datetime.today()).date()

# Data generation
records = []
for i in range(10000):
    emp_id = 100000000 + i
    country = random.choices(countries, weights=country_probs)[0]
    locale = country_locales[country]
    fake = faker_instances[locale]

    name = fake.name()
    phone = faker_us.phone_number()
    email = faker_us.email()
    gender = random.choices(genders, weights=gender_probs)[0]
    birthdate = generate_birthdate()
    hiredate = generate_hiredate(birthdate)

    department = random.choices(list(department_probs.keys()), weights=list(department_probs.values()))[0]
    role, (low, high) = random.choice(roles_by_dept[department])
    salary = random.randint(low, high)
    ssid = faker_us.ssn()

    records.append({
        "employeeID": emp_id,
        "CountryOfBirth": country,
        "name": name,
        "phone": phone,
        "email": email,
        "gender": gender,
        "birthdate": birthdate,
        "hiredate": hiredate,
        "department": department,
        "role": role,
        "salary": salary,
        "SSID": ssid
    })

# Create DataFrame and save
emp_df = pd.DataFrame(records)
emp_df.to_csv("emp_df.csv", index=False)
print("emp_df created with", len(emp_df), "rows and saved to emp_df.csv")


In [ ]:
emp_df.describe(include='all')

In [ ]:
emp_df.head(10)

In [ ]:
yearly_payroll = emp_df['salary'].sum()
print(f"The company pay in yearly payroll: ${yearly_payroll}")

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

count_of_birth = emp_df['CountryOfBirth'].value_counts()

plt.figure(figsize=(10, 5))
sns.barplot(x=count_of_birth.index, y=count_of_birth.values, palette="viridis")
plt.title('Employee Count by Country of Birth')
plt.xlabel('Country')
plt.ylabel('Number of Employees')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
count_of_employees = emp_df['department'].value_counts()

plt.figure(figsize=(10, 5))
sns.barplot(x=count_of_employees.index, y=count_of_employees.values, palette="viridis")
plt.title('Employee Count by Department')
plt.xlabel('Department')
plt.ylabel('Number of Employees')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:

emp_df['hiredate'] = pd.to_datetime(emp_df['hiredate'])

emp_df['hire_day'] = emp_df['hiredate'].dt.day_name()
hire_day_counts = emp_df['hire_day'].value_counts().reindex(
    ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
)

# Plot
plt.figure(figsize=(10, 5))
sns.barplot(x=hire_day_counts.index, y=hire_day_counts.values, color="purple")
plt.title('Number of Employees Hired on Each Day of the Week')
plt.xlabel('Day of the Week')
plt.ylabel('Number of Employees')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
emp_df['salary'] = pd.to_numeric(emp_df['salary'], errors='coerce')

plt.figure(figsize=(10, 5))
sns.kdeplot(data=emp_df, x="salary", color="#8A2BE2")
plt.title("KDE Plot of Employee Salaries")
plt.xlabel("Salary (USD)")
plt.ylabel("Density")
plt.tight_layout()
plt.show()

In [ ]:
emp_df['birthdate'] = pd.to_datetime(emp_df['birthdate'])
emp_df['birth_year'] = emp_df['birthdate'].dt.year

birth_year_counts = emp_df['birth_year'].value_counts().sort_index()

plt.figure(figsize=(10, 5))
plt.plot(birth_year_counts.index, birth_year_counts.values, marker='o', color="#8A2BE2")
plt.title("Number of Employees Born Each Year")
plt.xlabel("Year of Birth")
plt.ylabel("Number of Employees")
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
emp_df['salary'] = pd.to_numeric(emp_df['salary'], errors='coerce')

plt.figure(figsize=(10, 5))
sns.kdeplot(data=emp_df, x="salary", hue="department", common_norm=False)
plt.title("KDE Plot of Employee Salaries by Department")
plt.xlabel("Salary (USD)")
plt.ylabel("Density")
plt.tight_layout()
plt.show()

In [ ]:
print(emp_df.columns)


In [ ]:
import pandas as pd
import numpy as np

from datetime import datetime
current_year = datetime.now().year

emp_df['age'] = current_year - emp_df['birth_year']


weights = np.where((emp_df['age'] >= 40) & (emp_df['age'] <= 49), 3, 1)

smpl_df = emp_df.sample(n=500, weights=weights, random_state=42)


In [ ]:
print(smpl_df.describe(include='all'))

In [ ]:
print(smpl_df.head(10))

In [ ]:
import numpy as np

np.random.seed(42)

salary_std = emp_df['salary'].std()
noise = np.random.normal(loc=0, scale=0.1 * salary_std, size=emp_df.shape[0])

emp_df['perturbed_salary'] = emp_df['salary'] + noise

prtrb_df = emp_df.copy()
prtrb_df['salary'] = emp_df['perturbed_salary']
prtrb_df.drop(columns=['perturbed_salary'], inplace=True)

std_dev = emp_df['salary'].std() * 0.1


In [ ]:
print(prtrb_df.describe(include='all'))

In [ ]:
print(prtrb_df.head(10))

In [ ]:
!pip install ydata-profiling


In [ ]:
from ydata_profiling import ProfileReport
profile = ProfileReport(emp_df, title="Employee Data Profile", explorative=True)
profile.to_notebook_iframe()
